# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [17]:
import pandas as pd
import numpy as np

#Load the recipes file
recipes = pd.read_csv("recipes.tsv", sep="\t")

slugs = recipes['recipe_slug']

np.random.seed(42)  # for reproducibility
ratings = np.random.uniform(0.2, 0.8, size=len(slugs))  

ratings[:10] = 0.0  
ratings[10:20] = 1.0  


ratings_df = pd.DataFrame({
    'recipe_slug': slugs,
    'rating': ratings
})
ratings_df.to_csv("ratings.tsv", sep="\t", index=False)

print("ratings.tsv created with", len(ratings_df), "rows")


ratings.tsv created with 100 rows


## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [26]:
# YOUR CHANGES HERE

import pandas as pd

ratings = pd.read_csv("ratings.tsv", sep="\t")
tags    = pd.read_csv("recipe-tags.tsv", sep="\t")

ratings["slug_norm"] = ratings["recipe_slug"].astype(str).str.strip().str.lower()

slug_col = "recipe_slug" if "recipe_slug" in tags.columns else "slug"
tag_col  = "tag" if "tag" in tags.columns else "recipe_tag"
tags["slug_norm"] = tags[slug_col].astype(str).str.strip().str.lower()

X = pd.crosstab(tags["slug_norm"], tags[tag_col]).clip(0,1)
X = X.reindex(ratings["slug_norm"], fill_value=0).astype("int8")

features = pd.concat(
    [ratings["recipe_slug"].reset_index(drop=True),
     pd.Series(1.0, index=X.index, name="bias").reset_index(drop=True),
     X.reset_index(drop=True)],
    axis=1
)

features.to_csv("features.tsv", sep="\t", index=False)
print("features.tsv saved", features.shape)


features.tsv saved (100, 298)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [ ]:
# YOUR CHANGES HERE
import pandas as pd
from sklearn.linear_model import Ridge

# Load features and ratings
X = pd.read_csv("features.tsv", sep="\t")
y = pd.read_csv("ratings.tsv", sep="\t")["rating"]

# Drop recipe_slug to get only numeric features
X_num = X.drop(columns=["recipe_slug"])

# Fit ridge regression with alpha=1 and no intercept
model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X_num, y)

print("Coefficients (including bias):")
print(pd.Series(model.coef_, index=X_num.columns))


Coefficients (including bias):
bias            0.426626
alfredo         0.061913
almond         -0.046907
american        0.041269
appetizer      -0.049361
                  ...   
vegetarian     -0.082164
warm            0.018635
whippedcream    0.026235
winter         -0.081684
yeastdough     -0.068055
Length: 297, dtype: float64


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [ ]:
# YOUR CHANGES HERE
import pandas as pd
from sklearn.linear_model import Ridge

# Load data
X = pd.read_csv("features.tsv", sep="\t")
y = pd.read_csv("ratings.tsv",  sep="\t")["rating"]


X_num = X.drop(columns=["recipe_slug"]).astype(float)

model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X_num, y)

coef_df = pd.DataFrame({
    "recipe_tag": X_num.columns,
    "coefficient": model.coef_
})
coef_df.to_csv("model.tsv", sep="\t", index=False)
print("model.tsv saved", coef_df.shape)


model.tsv saved (297, 2)


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [21]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

# Load features and model
X = pd.read_csv("features.tsv", sep="\t")
coef = pd.read_csv("model.tsv", sep="\t").set_index("recipe_tag")["coefficient"]

# Align columns
feat_cols = [c for c in X.columns if c not in ("recipe_slug",)]
coef = coef.reindex(feat_cols).fillna(0.0)

# Compute score = dot(features, coefficients), preserving order
X_num = X[feat_cols].astype(float)
scores = X_num.values @ coef.values

# Save
out = pd.DataFrame({"recipe_slug": X["recipe_slug"], "score_estimate": scores})
out.to_csv("estimates.tsv", sep="\t", index=False)
print("estimates.tsv saved", out.shape)


estimates.tsv saved (100, 2)


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [22]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

# Inputs
Xdf = pd.read_csv("features.tsv", sep="\t")
ydf = pd.read_csv("ratings.tsv",  sep="\t")
wdf = pd.read_csv("model.tsv",    sep="\t")  

# Align columns
feat_cols = [c for c in Xdf.columns if c != "recipe_slug"]
X = Xdf[feat_cols].astype(float).values
y = ydf["rating"].astype(float).values
w = wdf.set_index("recipe_tag").reindex(feat_cols)["coefficient"].astype(float).values  # theta_hat

# Ridge parameters
lam = 1.0          # ridge (matches Part 3 alpha)
alpha_ucb = 2.0    # LinUCB alpha

# A = X^T X + λ I,  A_inv = (X^T X + λI)^{-1}
XT = X.T
A = XT @ X + lam * np.eye(X.shape[1])
A_inv = np.linalg.inv(A)

# Estimates (should match Part 4 if same model): X @ w
mu = X @ w

# Variance term per recipe: sqrt(x^T A_inv x)
# Efficiently compute diag(X A_inv X^T)
XAinv = X @ A_inv
var_term = np.sqrt(np.sum(XAinv * X, axis=1))

# Upper confidence bound
ucb = mu + alpha_ucb * var_term

# Save
out = pd.DataFrame({"recipe_slug": Xdf["recipe_slug"], "score_bound": ucb})
out.to_csv("bounds.tsv", sep="\t", index=False)
print("bounds.tsv saved", out.shape)


bounds.tsv saved (100, 2)


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [23]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

# Load data
Xdf = pd.read_csv("features.tsv", sep="\t")
ydf = pd.read_csv("ratings.tsv",  sep="\t")
feat_cols = [c for c in Xdf.columns if c != "recipe_slug"]

X = Xdf[feat_cols].astype(float).values       # (N, d)
y_map = dict(zip(Xdf["recipe_slug"], ydf["rating"].astype(float)))
slugs = Xdf["recipe_slug"].tolist()

# LinUCB params
lam = 1.0
alpha = 2.0
N, d = X.shape

# Initialize
A = lam * np.eye(d)        
b = np.zeros(d)           
A_inv = np.linalg.inv(A)   
recs = []

for t in range(100):
    theta = A_inv @ b                        
    # UCB for all arms
    XA = X @ A_inv                            
    var = np.sqrt(np.sum(XA * X, axis=1))     
    mu  = X @ theta
    p   = mu + alpha * var


    i = int(np.argmax(p))
    slug = slugs[i]
    x_i = X[i]
    r_i = float(y_map[slug])                 

    # Log before update (score_bound at selection time)
    recs.append((slug, float(p[i]), r_i))

   
    Ax = A_inv @ x_i
    denom = 1.0 + x_i @ Ax
    A_inv = A_inv - np.outer(Ax, Ax) / denom
    b = b + r_i * x_i

# Save recommendations
out = pd.DataFrame(recs, columns=["recipe_slug", "score_bound", "reward"])
out.to_csv("recommendations.tsv", sep="\t", index=False)
print("recommendations.tsv saved", out.shape)


recommendations.tsv saved (100, 3)


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [24]:
ack_text = """Acknowledgments

I used ChatGPT as a study partner to clarify  and check errors, and simplify code while completing this project. 


For transparency, I am including links to the transcripts of these interactions:

1. ChatGPT guidance: [https://chatgpt.com/share/68d20a8d-0870-8007-bb79-080efab54c33]
These resources document the support I received and ensure compliance with the class generative AI policy.
"""

with open("acknowledgments.txt","w") as f:
    f.write(ack_text)


## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.